In [ ]:
#scraper to take links collected by LibraryThingScraper
#return scrape of website

In [173]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np
import requests
import re

In [264]:
def GetBookPlease(page):
    
    #print(page)
    series = 0
        
    #get response
    response = requests.get(url='https://www.librarything.com'+ page).text
    soup = BeautifulSoup(response, features="html.parser")
    
    #get top of page - author, publish date, if part of series
    top = soup.find(attrs={'class':'headsummary'})
    published = top.find(attrs={'class':'date'})
    if published != None:
        published = top.find(attrs={'class':'date'}).string
    author = top.find('h2')
    if author != None:
        author = top.find('h2').text
    stuff = top.find_all('h3')                          #this will always exist
    for e in stuff:
        if 'Series:' in e.text:
            series = 1
    
    #get rating
    rating = soup.find(attrs={'class':'dark_hint'})
    if rating != None:
        rating = soup.find(attrs={'class':'dark_hint'}).text
        
    #get number of characters
    groups = soup.find_all(attrs={'class':'fwikiGroup'})
    for g in range(len(groups)):
        hm = groups[g].find(attrs={'fieldname':'characternames'})
        if hm is not None:
            place = g
    charnum = groups[place].find(attrs={'class':'itemnumberoverflow'})
    if charnum is None:
        charnum = groups[place].find_all(attrs={'class':'fwikiAtomicValue'})
        charnum = len(charnum)
    else:
        charnum = groups[place].find(attrs={'class':'itemnumberoverflow'}).text
        charnum = re.findall(r'\d+', charnum)[0]

    #get number of awards
    #would be nice to have titles but not enough time
    for g in range(len(groups)):
        hm = groups[g].find(attrs={'fieldname':'awards'})
        if hm is not None:
            place = g
    awardnum = groups[place].find(attrs={'class':'itemnumberoverflow'})
    if awardnum is None:
        awardnum = groups[place].find_all(attrs={'class':'fwikiAtomicValue'})
        awardnum = len(awardnum)
    else:
        awardnum = groups[place].find(attrs={'class':'itemnumberoverflow'}).text
        awardnum = re.findall(r'\d+', awardnum)[0]
        
    #NOTE: this will still return values if the link is for a movie
    return [page, published, author, series, rating, charnum, awardnum]


In [258]:
#pull in the link list
moviesdb = pd.read_csv('LTlinksFull.csv')

#make a list of book links
links = moviesdb['Link2']

#if there is no Link2 AND the last search term did not have ] in it, try Link1
for m in range(len(links)):
    if str(links[m])=='nan':
        if r']' not in moviesdb.loc[m, 'SearchTerm']:
            #print(moviesdb.loc[m, 'Link2'])
            links[m] = moviesdb.loc[m, 'Link1']

In [263]:
#batch maker
links = links[576:].reset_index(drop=True)
#print(links)

In [260]:
library = pd.DataFrame(columns = ['link','published', 'author', 'series', 'rating', 'charnum', 'awardnum'])
#links = ['/work/4041453', '/work/1906740', '/work/4725', '/work/5716']

for l in range(len(links)):
    if str(links[l]) != 'nan':
        grab = GetBookPlease(links[l])
        library = library.append(pd.Series(grab, index = library.columns), ignore_index=True)

#print(library)

/work/582643
/work/5042572
/work/2071522
/work/63568
/work/2742161
/work/435532
/work/474221
/work/19751569
/work/8144098
/work/12502
/work/1748
/work/8696187
/work/2848470
/work/5196084
/work/27987
/work/21622
/work/3992264
/work/74629
/work/6378
/work/12989
/work/5708
/work/4156037
/work/861116
/work/31129
/work/45136
/work/71874
/work/87340
/work/10168
/work/732079
/work/4936
/work/2311
/work/2725913
/work/123771
/work/3779194
/work/4286751
/work/15669521
/work/13114
/work/187363
/work/98513
/work/25867
/work/2254626
/work/283438
/work/8053033
/work/19061895
/work/72154
/work/19061841
/work/25932475
/work/3203356
/work/419862
/work/13576
/work/191408
/work/7918
/work/6621
/work/2421
/work/1638
/work/6252
/work/6999
/work/59888
/work/33070
/work/450701
/work/132896
/work/318194
/work/8362
/work/8196
/work/249233
/work/4323
/work/18249
/work/3659
/work/4300
/work/171336
/work/11577
/work/7288
/work/71596
/work/16702962
/work/28377
/work/15659
/work/5091462
/work/9802440
/work/214511
/

/work/43286
/work/393681
/work/2573519
/work/1562126
/work/12504
/work/214526
/work/9214710
/work/101173
/work/60695
/work/23117
/work/12286134
/work/16902059
/work/257339
/work/90655
/work/24873280
/work/2554470
/work/6050
/work/885007
/work/5867
/work/1811968
/work/2661011
/work/18244
/work/433884
/work/12521801
/work/1131906
/work/24444777
/work/14784289
/work/303025
/work/748566
/work/10762830
/work/1362699
/work/62213
/work/2374090
/work/1817318
/work/7189
/work/4915
/work/3765930
/work/10740649
/work/10187
/work/17833941
/work/92653
/work/2970534
/work/4607155
/work/29995
/work/1386651
/work/20410
/work/4809
/work/15930328
/work/15774456
/work/25267351
/work/558221
/work/151679
/work/38539
/work/11744
/work/22341293
/work/308736
/work/1002505
/work/2308923
/work/12286130
/work/12444
/work/299323
/work/27188
/work/4055967
/work/24251
/work/2122
/work/113803
/work/69475
/work/4807
/work/429502
/work/8387386
/work/9784168
/work/690192
/work/4808
/work/108681
/work/21919175
/work/296

In [262]:
print(library.info())
library.to_csv('LTScrape3.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   link       748 non-null    object
 1   published  512 non-null    object
 2   author     746 non-null    object
 3   series     748 non-null    object
 4   rating     710 non-null    object
 5   charnum    748 non-null    object
 6   awardnum   748 non-null    object
dtypes: object(7)
memory usage: 41.0+ KB
None


In [266]:
#this cell is just to slap everything together since I had trouble running the full scrape
one = pd.read_csv('LTScrape1.csv')
two = pd.read_csv('LTScrape2.csv')
three = pd.read_csv('LTScrape3.csv')

together = pd.concat([one,two,three])
#together = together.applymap(str)
together.drop_duplicates(inplace=True, ignore_index=True)

print(together.info())

together.to_csv('LTScrapeFull.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269 entries, 0 to 1268
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   link       1269 non-null   object
 1   published  832 non-null    object
 2   author     1267 non-null   object
 3   series     1269 non-null   int64 
 4   rating     1213 non-null   object
 5   charnum    1269 non-null   int64 
 6   awardnum   1269 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 69.5+ KB
None
